# PhobiaShield - Complete Training Pipeline
**The Architect Module**

1. Download mini dataset
2. Train PhobiaNet
3. Test on real data

**Setup:** Runtime > GPU (T4)

In [ ]:
# Setup - Clone repo
!git clone https://github.com/Gabriele-mp/PhobiaShield.git
%cd PhobiaShield
!git checkout TheArchitect
!pip install -q -e .

In [ ]:
# GPU Check
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f}GB")

## Step 1: Download Mini Dataset

In [ ]:
# Download COCO mini dataset
!python scripts/download_mini_dataset.py

## Step 2: Quick Test Training (5 epochs)

In [ ]:
# Fast test training
!python scripts/train_complete.py training=fast_test

## Step 3: Full Training (50 epochs)

In [ ]:
# Full training (uncomment to run)
# !python scripts/train_complete.py training.epochs=50

## Step 4: Test on Real Images

In [ ]:
# Load trained model and test
from omegaconf import OmegaConf
from src.models.phobia_net import PhobiaNet
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from pathlib import Path

# Load model
model_cfg = OmegaConf.load('cfg/model/tiny_yolo.yaml')
model = PhobiaNet(model_cfg).to(device)

# Load best weights
checkpoint = torch.load('outputs/checkpoints/best_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print("✓ Model loaded")

In [ ]:
# Test on random validation images
import torchvision.transforms as T
import cv2

# Get test images
test_images = list(Path('data/mini_dataset/test/images').glob('*.jpg'))[:6]

# Transform
transform = T.Compose([
    T.Resize((416, 416)),
    T.ToTensor(),
])

# Class names
class_names = ['Clown', 'Shark', 'Spider']

# Plot results
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, img_path in enumerate(test_images):
    # Load image
    img = Image.open(img_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)
    
    # Predict
    with torch.no_grad():
        predictions = model.predict(img_tensor, conf_threshold=0.5)
    
    # Draw
    img_np = np.array(img)
    
    for pred in predictions[0]:  # First image in batch
        x, y, w, h = pred['bbox']
        conf = pred['confidence']
        cls = pred['class_id']
        
        # Convert to pixel coords
        h_img, w_img = img_np.shape[:2]
        x1 = int((x - w/2) * w_img)
        y1 = int((y - h/2) * h_img)
        x2 = int((x + w/2) * w_img)
        y2 = int((y + h/2) * h_img)
        
        # Draw bbox
        cv2.rectangle(img_np, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img_np, f"{class_names[cls]} {conf:.2f}", 
                   (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    axes[idx].imshow(img_np)
    axes[idx].axis('off')
    axes[idx].set_title(f"Detections: {len(predictions[0])}")

plt.tight_layout()
plt.show()

print("✅ Inference complete!")

## Step 5: Performance Metrics

In [ ]:
# Calculate mAP on test set
from src.training.metrics import calculate_map
from src.data.phobia_dataset import PhobiaDataset
from torch.utils.data import DataLoader

# Test dataset
test_dataset = PhobiaDataset(
    img_dir='data/mini_dataset/test/images',
    label_dir='data/mini_dataset/test/labels',
    img_size=416,
    grid_size=13,
    num_boxes=2,
    num_classes=3,
    augment=False
)

test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Calculate metrics
print("Calculating mAP on test set...")
map_score = calculate_map(model, test_loader, device, iou_threshold=0.5)
print(f"\nmAP@0.5: {map_score:.4f}")
print("\n✅ All done!")